In [1]:
# importing lib

import pandas as pd

import os

import re

In [6]:
# adjusting display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [2]:
# grabbing cwd

cwd = os.getcwd()

cwd

'C:\\Users\\CharlesYi\\Jupyter Notebook\\Alteryx Challenges\\Challenge 188_Difficult, but not im-parse-able'

In [4]:
# joining file name to path

file_name = 'data.xlsx'

file_path = os.path.join(cwd, file_name)

file_path

'C:\\Users\\CharlesYi\\Jupyter Notebook\\Alteryx Challenges\\Challenge 188_Difficult, but not im-parse-able\\data.xlsx'

In [7]:
# reading data

df = pd.read_excel(file_path)

df.head()

,Field_1
0,----------------------------------------------------------------------------------------------------
1,(21)( 1 )Pag.
2,NaN
3,NaN
4,NaN


In [15]:
# skipping first 46 rows

df = df.iloc[46:]

df.head()

,Field_1
46,| 1 |191-01-77441|#.####> | 1 | | | | | | |2|2|1|6|4|0|0|0| | | | | | |2|2|1|6|4|0|0 | |
47,----------------------------------------------------------------------------------------------------
48,| Doc number Type Post date Document Dt Base amt Tax Code Tax amt |
49,----------------------------------------------------------------------------------------------------
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |"


In [20]:
# parsing VAT no

df['vat_no'] = df['Field_1'].apply(lambda x: re.findall(r'^\|\s+\d+\s+\|(.+?)\|.+?', str(x)))

df.head()

,Field_1,vat_no
46,| 1 |191-01-77441|#.####> | 1 | | | | | | |2|2|1|6|4|0|0|0| | | | | | |2|2|1|6|4|0|0 | |,[191-01-77441]
47,----------------------------------------------------------------------------------------------------,[]
48,| Doc number Type Post date Document Dt Base amt Tax Code Tax amt |,[]
49,----------------------------------------------------------------------------------------------------,[]
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |",[]


In [23]:
# function to extract element from list

def list_converter(row):
    
    if row:
        return row[0]
    else:
        return None

In [26]:
# applying function

df['vat_no_parsed'] = df['vat_no'].apply(list_converter)

df.head()

,Field_1,vat_no,vat_no_parsed
46,| 1 |191-01-77441|#.####> | 1 | | | | | | |2|2|1|6|4|0|0|0| | | | | | |2|2|1|6|4|0|0 | |,[191-01-77441],191-01-77441
47,----------------------------------------------------------------------------------------------------,[],None
48,| Doc number Type Post date Document Dt Base amt Tax Code Tax amt |,[],None
49,----------------------------------------------------------------------------------------------------,[],None
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |",[],None


In [28]:
# creating function to replace none in vat_no

vat_no_filled = []

for index, row in df.iterrows():
    
    if pd.isna(row['vat_no_parsed']):
        vat_no_filled.append(vat_no_filled[-1])
    else:
        vat_no_filled.append(row['vat_no_parsed'])

In [30]:
# inserting list as col

df['vat_no_parsed_filled'] = vat_no_filled

df.head(8)

,Field_1,vat_no,vat_no_parsed,vat_no_parsed_filled
46,| 1 |191-01-77441|#.####> | 1 | | | | | | |2|2|1|6|4|0|0|0| | | | | | |2|2|1|6|4|0|0 | |,[191-01-77441],191-01-77441,191-01-77441
47,----------------------------------------------------------------------------------------------------,[],None,191-01-77441
48,| Doc number Type Post date Document Dt Base amt Tax Code Tax amt |,[],None,191-01-77441
49,----------------------------------------------------------------------------------------------------,[],None,191-01-77441
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |",[],None,191-01-77441
51,----------------------------------------------------------------------------------------------------,[],None,191-01-77441
52,| 2 |192-82-04066|#### #> | 4 | | | | |5|7|7|2|6|8|0|4|9|8| | | | |5|7|7|2|6|8|0|4|9 | |,[192-82-04066],192-82-04066,192-82-04066
53,----------------------------------------------------------------------------------------------------,[],None,192-82-04066


In [34]:
# parsing additional fields

df['other_parsed'] = df['Field_1'].apply(lambda x: re.findall(r'^\| (\d+)\s+' # doc number
                                         r'([A-Z]{2})\s+' # type
                                         r'(\d{4}-\d{2}-\d{2})\s+' # post date
                                         r'(\d{4}-\d{2}-\d{2})\s+' # document dt
                                         r'(.+?)\s+' # base amt
                                         r'([A-Z]\d)\s+' # tax code
                                         r'(.+?)\s+\|$', # tax amt
                                         str(x)))

df.head()

,Field_1,vat_no,vat_no_parsed,vat_no_parsed_filled,other_parsed
46,| 1 |191-01-77441|#.####> | 1 | | | | | | |2|2|1|6|4|0|0|0| | | | | | |2|2|1|6|4|0|0 | |,[191-01-77441],191-01-77441,191-01-77441,[]
47,----------------------------------------------------------------------------------------------------,[],None,191-01-77441,[]
48,| Doc number Type Post date Document Dt Base amt Tax Code Tax amt |,[],None,191-01-77441,[]
49,----------------------------------------------------------------------------------------------------,[],None,191-01-77441,[]
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |",[],None,191-01-77441,"[(1900257027, KR, 2019-06-23, 2019-06-30, 22,164,000, P1, 2,216,400)]"


In [35]:
# checking dtype of other_parsed

df['other_parsed'].apply(type)

46     <class 'list'>
47     <class 'list'>
48     <class 'list'>
49     <class 'list'>
50     <class 'list'>
            ...      
180    <class 'list'>
181    <class 'list'>
182    <class 'list'>
183    <class 'list'>
184    <class 'list'>
Name: other_parsed, Length: 139, dtype: object

In [36]:
# checking dtype of elements within list

df['other_parsed'].apply(lambda x: [type(i) for i in x])

46                    []
47                    []
48                    []
49                    []
50     [<class 'tuple'>]
             ...        
180    [<class 'tuple'>]
181    [<class 'tuple'>]
182    [<class 'tuple'>]
183    [<class 'tuple'>]
184                   []
Name: other_parsed, Length: 139, dtype: object

In [39]:
# extracting tuple to strings

df['other_parsed_list'] = df['other_parsed'].apply(lambda x: x[0] if x else None)

df.head()

,Field_1,vat_no,vat_no_parsed,vat_no_parsed_filled,other_parsed,other_parsed_list
46,| 1 |191-01-77441|#.####> | 1 | | | | | | |2|2|1|6|4|0|0|0| | | | | | |2|2|1|6|4|0|0 | |,[191-01-77441],191-01-77441,191-01-77441,[],None
47,----------------------------------------------------------------------------------------------------,[],None,191-01-77441,[],None
48,| Doc number Type Post date Document Dt Base amt Tax Code Tax amt |,[],None,191-01-77441,[],None
49,----------------------------------------------------------------------------------------------------,[],None,191-01-77441,[],None
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |",[],None,191-01-77441,"[(1900257027, KR, 2019-06-23, 2019-06-30, 22,164,000, P1, 2,216,400)]","(1900257027, KR, 2019-06-23, 2019-06-30, 22,164,000, P1, 2,216,400)"


In [43]:
# splitting other_parsed_list to columns

df[['doc_number', 'type', 'post_date', 'document_dt', 'base_amt', 'tax_code', 'tax_amt']] = df['other_parsed_list'].apply(lambda x: pd.Series(x))

df.head()

,Field_1,vat_no,vat_no_parsed,vat_no_parsed_filled,other_parsed,other_parsed_list,doc_number,type,post_date,document_dt,base_amt,tax_code,tax_amt
46,| 1 |191-01-77441|#.####> | 1 | | | | | | |2|2|1|6|4|0|0|0| | | | | | |2|2|1|6|4|0|0 | |,[191-01-77441],191-01-77441,191-01-77441,[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,----------------------------------------------------------------------------------------------------,[],None,191-01-77441,[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,| Doc number Type Post date Document Dt Base amt Tax Code Tax amt |,[],None,191-01-77441,[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,----------------------------------------------------------------------------------------------------,[],None,191-01-77441,[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |",[],None,191-01-77441,"[(1900257027, KR, 2019-06-23, 2019-06-30, 22,164,000, P1, 2,216,400)]","(1900257027, KR, 2019-06-23, 2019-06-30, 22,164,000, P1, 2,216,400)",1900257027,KR,2019-06-23,2019-06-30,"22,164,000",P1,"2,216,400"


In [49]:
# filtering out where doc number is null

df_filtered = df[~pd.isna(df['doc_number'])]

df_filtered.head()

,Field_1,vat_no,vat_no_parsed,vat_no_parsed_filled,other_parsed,other_parsed_list,doc_number,type,post_date,document_dt,base_amt,tax_code,tax_amt
50,"| 1900257027 KR 2019-06-23 2019-06-30 22,164,000 P1 2,216,400 |",[],None,191-01-77441,"[(1900257027, KR, 2019-06-23, 2019-06-30, 22,164,000, P1, 2,216,400)]","(1900257027, KR, 2019-06-23, 2019-06-30, 22,164,000, P1, 2,216,400)",1900257027,KR,2019-06-23,2019-06-30,"22,164,000",P1,"2,216,400"
56,"| 1900406000 KR 2019-06-07 2019-06-13 4,250,000,000 P1 425,000,000 |",[],None,192-82-04066,"[(1900406000, KR, 2019-06-07, 2019-06-13, 4,250,000,000, P1, 425,000,000)]","(1900406000, KR, 2019-06-07, 2019-06-13, 4,250,000,000, P1, 425,000,000)",1900406000,KR,2019-06-07,2019-06-13,"4,250,000,000",P1,"425,000,000"
57,"| 1900013009 KR 2019-06-28 2019-06-30 85,038,400 P2 8,503,840 |",[],None,192-82-04066,"[(1900013009, KR, 2019-06-28, 2019-06-30, 85,038,400, P2, 8,503,840)]","(1900013009, KR, 2019-06-28, 2019-06-30, 85,038,400, P2, 8,503,840)",1900013009,KR,2019-06-28,2019-06-30,"85,038,400",P2,"8,503,840"
58,"| 1800256000 KR 2019-06-26 2019-06-30 556,479,040 P1 55,647,904 |",[],None,192-82-04066,"[(1800256000, KR, 2019-06-26, 2019-06-30, 556,479,040, P1, 55,647,904)]","(1800256000, KR, 2019-06-26, 2019-06-30, 556,479,040, P1, 55,647,904)",1800256000,KR,2019-06-26,2019-06-30,"556,479,040",P1,"55,647,904"
59,"| 1900268001 KR 2019-06-29 2019-06-30 881,163,058 P1 88,116,305 |",[],None,192-82-04066,"[(1900268001, KR, 2019-06-29, 2019-06-30, 881,163,058, P1, 88,116,305)]","(1900268001, KR, 2019-06-29, 2019-06-30, 881,163,058, P1, 88,116,305)",1900268001,KR,2019-06-29,2019-06-30,"881,163,058",P1,"88,116,305"


In [63]:
# selecting relevant cols

df_final = df_filtered[['vat_no_parsed_filled', 'post_date', 'doc_number', 'base_amt', 'tax_amt', 'tax_code', 'type', 'document_dt']]

df_final.head()

,vat_no_parsed_filled,post_date,doc_number,base_amt,tax_amt,tax_code,type,document_dt
50,191-01-77441,2019-06-23,1900257027,"22,164,000","2,216,400",P1,KR,2019-06-30
56,192-82-04066,2019-06-07,1900406000,"4,250,000,000","425,000,000",P1,KR,2019-06-13
57,192-82-04066,2019-06-28,1900013009,"85,038,400","8,503,840",P2,KR,2019-06-30
58,192-82-04066,2019-06-26,1800256000,"556,479,040","55,647,904",P1,KR,2019-06-30
59,192-82-04066,2019-06-29,1900268001,"881,163,058","88,116,305",P1,KR,2019-06-30


In [64]:
# filtering to p1 or p9

df_final = df_final[(df_final['tax_code'] == 'P1') | (df_final['tax_code'] == 'P9')]

df_final.head()

,vat_no_parsed_filled,post_date,doc_number,base_amt,tax_amt,tax_code,type,document_dt
50,191-01-77441,2019-06-23,1900257027,"22,164,000","2,216,400",P1,KR,2019-06-30
56,192-82-04066,2019-06-07,1900406000,"4,250,000,000","425,000,000",P1,KR,2019-06-13
58,192-82-04066,2019-06-26,1800256000,"556,479,040","55,647,904",P1,KR,2019-06-30
59,192-82-04066,2019-06-29,1900268001,"881,163,058","88,116,305",P1,KR,2019-06-30
65,204-71-37125,2019-06-29,1904013007,"51,812,290","5,181,229",P1,KR,2019-06-30


In [66]:
# sort by vat_no and document dt

df_final.sort_values(by = ['vat_no_parsed_filled', 'document_dt'])

,vat_no_parsed_filled,post_date,doc_number,base_amt,tax_amt,tax_code,type,document_dt
150,111-81-41019,2019-06-30,1910007001,"6,960,950","696,095",P1,KR,2019-06-17
152,111-81-41019,2019-06-30,1910261006,"14,626,760","1,462,676",P1,KR,2019-07-01
155,111-81-41019,2019-06-30,5110466063,"102,625,940","10,262,594",P1,RE,2019-07-01
151,111-81-41019,2019-06-30,1910026000,"15,485,620","1,548,562",P1,KR,2019-07-12
153,111-81-41019,2019-06-30,5110006019,"96,541,270","9,654,127",P1,RE,2019-07-12
154,111-81-41019,2019-06-30,5110029001,"104,791,530","10,479,153",P1,RE,2019-07-25
144,116-85-36080,2019-06-16,5140482033,"2,640,000","264,000",P1,RE,2019-07-01
109,117-86-38534,2019-06-16,5800476217,"840,000","84,000",P1,RE,2019-06-01
107,117-86-38534,2019-06-01,5160429030,"56,000","5,600",P1,RE,2019-06-26
108,117-86-38534,2019-06-24,5148466117,"60,000","6,000",P1,RE,2019-06-29
